In [150]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.features import shapes
import rioxarray
import matplotlib.pyplot as plt
import numpy as np
import warnings
from skimage.segmentation import slic
from shapely.geometry import shape
from rasterstats import zonal_stats

from gridflow import model
from gridflow.visuals import country_viz
from gridflow import utils
from gridflow import epm_input_generator
from gridflow import testing
import pycountry

def country_to_iso3(country_name):
    try:
        country = pycountry.countries.lookup(country_name)
        return country.alpha_3
    except LookupError:
        return None  # Not found

In [169]:
from gridflow.testing import *

In [167]:
countries = ["LUX"]
lux = model.region(countries, 'data/global_datasets')
lux.create_zones(n=5)
epm_input_generator.zone_replicate(lux, "data/test/epm_inputs_raw/load/pDemandProfile.csv","data/test/epm_inputs_raw/load/pDemandProfile.csv")

UnboundLocalError: cannot access local variable 'df' where it is not associated with a value

In [164]:
import pandas as pd

#edited version of old replicator
def one_zone_replicate(df_original, zone_ids):
    

    if 'zone' in df_original.columns:
        # Replicate data for each subregion
        df_final = pd.concat([
            df_original.assign(zone=zone_id) 
            for zone_id in zone_ids
        ], ignore_index=True)
        
        
    else:
        raise ValueError("The input data is not zonal.")
    
    return df_final



#setting up fake datasets
df_testing = pd.DataFrame({
    "zone": ["TUR", "TUR", "TUR", "SYR", "SYR", "SYR"],
    "type": ["peak", "avg", "?", "peak", "avg", "?"],
    "number": [25, 30, 35, 40, 45, 50],
    "other number": [0.5,0.6,0.9,0.5,0.2,0.1]

})
print("Input data")
print (df_testing)
zone_df = pd.DataFrame({
    "geometry" : [None, None, None, None],
    "country" : ["TUR","TUR","SYR", "SYR"]})

#Stuff starts here
dffinal = pd.DataFrame()
print ("\nInput zones")
print(zone_df)

while not df_testing.empty:
    chosen = df_testing.loc[0,"zone"]
    newdf = (df_testing[df_testing["zone"] == chosen])
    df_testing = df_testing[df_testing["zone"] != chosen]
    df_testing = df_testing.reset_index(drop=True)
    
    country_zones = []
    #print (zone_df)
    #print (len(zone_df))
    for i in range (len(zone_df)):
            if chosen in zone_df.loc[i,"country"]:
                country_zones.append(zone_df.loc[i,"country"]+str(i))
    
    
    newdf = one_zone_replicate(newdf, country_zones)
    dffinal = pd.concat([dffinal, newdf], ignore_index=True)
print("\nOutput")
print(dffinal)

Input data
  zone  type  number  other number
0  TUR  peak      25           0.5
1  TUR   avg      30           0.6
2  TUR     ?      35           0.9
3  SYR  peak      40           0.5
4  SYR   avg      45           0.2
5  SYR     ?      50           0.1

Input zones
  geometry country
0     None     TUR
1     None     TUR
2     None     SYR
3     None     SYR

Output
    zone  type  number  other number
0   TUR0  peak      25           0.5
1   TUR0   avg      30           0.6
2   TUR0     ?      35           0.9
3   TUR1  peak      25           0.5
4   TUR1   avg      30           0.6
5   TUR1     ?      35           0.9
6   SYR2  peak      40           0.5
7   SYR2   avg      45           0.2
8   SYR2     ?      50           0.1
9   SYR3  peak      40           0.5
10  SYR3   avg      45           0.2
11  SYR3     ?      50           0.1
